Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 
import re
from nltk import pos_tag

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
# Напишем функцию для лемматиизации и применим её для нашего датафрейма
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
data['lemma_text'] = data['text'].apply(lemmatize_text)
data['lemma_text'] = data['lemma_text'].apply(" ".join)
# Стоит учесть, что лемматизация не учитывает части речи. 
# Для этого используют nltk.pos_tag. 
# Однако, пока что, использование pos_tag длиться несколько часов, а затем и вообще крашится 

In [4]:
# Очистим от лишних символов текст
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return " ".join(text.split())
data['lemma_text'] = data['lemma_text'].apply(clear_text)

In [5]:
data.head()

,text,toxic,lemma_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [6]:
train, test = train_test_split(data, test_size = 0.5, random_state = 12345)
train.shape, test.shape

((79785, 3), (79786, 3))

In [7]:
corpus = train['lemma_text'].values.astype('U')

In [8]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(sublinear_tf = True, stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

In [9]:
target_train = train['toxic'].values.astype('U')
target_test = test['toxic'].values.astype('U')

In [10]:
features_test = count_tf_idf.transform(test['lemma_text'])

# 2. Обучение

### Логистическая регрессия

In [11]:
# Проверим влияние "параметров" на качество модели логистической регрессии
lr = LogisticRegression(random_state = 12345)
parameters_lr_1 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l1'], # Регуляризация 
                 'solver':['liblinear', 'saga'],
                 'class_weight':[None, 'balanced']} # функция потерь
parameters_lr_2 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l2'], # Регуляризация 
                 'solver':['lbfgs', 'saga', 'sag', 'newton-cg'], 
                 'class_weight':[None, 'balanced']} # функция потерь

search_lr_1 = RandomizedSearchCV(lr, parameters_lr_1, cv=5, n_jobs = -1)
search_lr_1.fit(tf_idf, target_train)
best_lr_1 = search_lr_1.best_estimator_
predict_lr_1 = best_lr_1.predict(features_test)

search_lr_2 = RandomizedSearchCV(lr, parameters_lr_2, cv=5, n_jobs = -1)
search_lr_2.fit(tf_idf, target_train)
best_lr_2 = search_lr_2.best_estimator_
predict_lr_2 = best_lr_2.predict(features_test)

print('F1 для Logistic Regression (регуляризация l1) на тестовой выборке:', f1_score(target_test, predict_lr_1, pos_label='1'))
print()
print('F1 для Logistic Regression (регуляризация l2) на тестовой выборке:', f1_score(target_test, predict_lr_2, pos_label='1'))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


F1 для Logistic Regression (регуляризация l1) на тестовой выборке: 0.7810453147974871

F1 для Logistic Regression (регуляризация l2) на тестовой выборке: 0.7628822197055493


### SGD

In [15]:
sgd = SGDClassifier(random_state = 12345)
parameters_sgd = {'penalty':['l1', 'l2', 'elasticnet'], # регуляризация
                  'alpha':[0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1], # коэффициент альфа
                  'max_iter':range(100, 1500, 200), # максимальное количество итераций
                  'early_stopping': [True, False], # ранняя остановка градиентного спуска
                  'class_weight':[None, 'balanced']} # балансировка классов
search_sgd = RandomizedSearchCV(sgd, parameters_sgd, cv=5, n_jobs = -1)
search_sgd.fit(tf_idf, target_train)
best_sgd = search_sgd.best_estimator_
predict_sgd = best_sgd.predict(features_test)
print('F1 для SGD на тестовой выборке:', f1_score(target_test, predict_sgd, pos_label='1'))

F1 для SGD на тестовой выборке: 0.7823844744520595


# 3. Выводы

По полученным результатам можно сделать вывод, что наилучшей моделью является SGDClassifier, настроенной так, что f1 score больше 0.75 (~0.782). Для достижения этого результата были применены лемматизация и очистка от лишних символов.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны